In [82]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
%matplotlib inline

In [83]:
import geocoder
import requests
import time

In [84]:
pd.options.display.max_columns = None
display(HTML("<style>.container { width:100% !important; }</style>"))

In [5]:
CSV_PATH = os.path.join('data', 'hacknight_ticket_sample_data_2015.csv')
df = pd.read_csv(CSV_PATH,low_memory=False, parse_dates=['issue_date', 'ticket_queue_date'])

In [6]:
df.head()

,ticket_number,issue_date,violation_location,license_plate_number,license_plate_state,license_plate_type,zipcode,violation_code,violation_description,unit,unit_description,vehicle_make,fine_level1_amount,fine_level2_amount,current_amount_due,total_payments,ticket_queue,ticket_queue_date,notice_level,hearing_disposition,notice_number,officer,address
0,9188814621,2015-05-07 13:52:00,2134 S ARCHER AV,7c189a16ef79db9413c1f46b7e5d1712e5c0c1575be352...,MI,PAS,48103,0964190A,EXP. METER NON-CENTRAL BUSINESS DISTRICT,498,DOF,BUIC,50,100,0.0,50.0,Paid,2015-05-13,NaN,NaN,0,798,"2100 s archer av, chicago, il"
1,65543631,2015-01-10 17:09:00,423 N LARAMIE,13d93046fb2c3bc2d79baa8647d620d0c5a65fdc223cb9...,IL,PAS,606394357,0976160F,EXPIRED PLATES OR TEMPORARY REGISTRATION,15,CPD,CHRY,60,120,146.4,0.0,Notice,2015-01-28,SEIZ,NaN,5109291290,16391,"400 n laramie, chicago, il"
2,9188975267,2015-06-11 18:49:00,1338 W CORNELIA,265ee87ad9d24c7732f65383e6b9af1f344348bf639ce2...,IL,PAS,606134985,0976160F,EXPIRED PLATES OR TEMPORARY REGISTRATION,502,DOF,BMW,60,120,0.0,60.0,Paid,2015-06-23,VIOL,NaN,5190301770,1444,"1300 w cornelia, chicago, il"
3,64465302,2015-01-14 08:09:00,229 W CERMAK RD,2800f6eaba1dc6fde0b0785df17c808dba871d1570cc4c...,IL,PAS,605442187,0964130A,PARK ALLEY,495,Miscellaneous,HOND,150,300,0.0,150.0,Paid,2015-04-28,DETR,NaN,5191229940,313,"200 w cermak rd, chicago, il"
4,9189537354,2015-10-28 16:26:00,2410 N MONTICELLO AV,f0858c838c8cabda4e8fd3e0425a6c330a76e006f9de01...,IL,PAS,60625,0964190A,EXP. METER NON-CENTRAL BUSINESS DISTRICT,729,Miscellaneous,TOYT,50,100,0.0,100.0,Paid,2016-01-27,SEIZ,NaN,5168404170,1407,"2400 n monticello av, chicago, il"


In [7]:
df['address'].head()

0        2100 s archer av, chicago, il
1           400 n laramie, chicago, il
2         1300 w cornelia, chicago, il
3         200 w cermak rd, chicago, il
4    2400 n monticello av, chicago, il
Name: address, dtype: object

In [8]:
addrs_df = pd.DataFrame(df['address'].str.split(',').tolist(), columns=['address', 'city', 'state'])
addrs_df.head()

,address,city,state
0,2100 s archer av,chicago,il
1,400 n laramie,chicago,il
2,1300 w cornelia,chicago,il
3,200 w cermak rd,chicago,il
4,2400 n monticello av,chicago,il


In [9]:
addrs_df['city'].unique()

array([' chicago'], dtype=object)

In [10]:
addrs_df['state'].unique()

array([' il'], dtype=object)

Ok, from these lists of unique values, we see the 'address' field was very clean and consistent. As we expect, the city was always Chicago and the state was always IL. 

Looking at the street numbers, it looks like addresses were truncated down to just their block.  From the count of unique addresses below, we see that on average, each address in this sample of the data got about 2.5 tickets.  As geocoding addresses is an expensive operation, I'll only geocode each address once, and then use an inner join later to add in latitude and longitude values for all addresses. 

In [11]:
print('Total number of entried:    {}'.format(len(addrs_df['address'])))
print('Number of unique addresses: {}'.format(len(addrs_df['address'].unique())))

Total number of entried:    50000
Number of unique addresses: 20608


In [12]:
addrs_df = pd.DataFrame(addrs_df['address'].unique(), columns=['address'])
addrs_df['lat'] = np.nan
addrs_df['lng'] = np.nan
addrs_df.head()

,address,lat,lng
0,2100 s archer av,NaN,NaN
1,400 n laramie,NaN,NaN
2,1300 w cornelia,NaN,NaN
3,200 w cermak rd,NaN,NaN
4,2400 n monticello av,NaN,NaN


In [69]:
class GeoSessions:
    def __init__(self):
        self.Arcgis = requests.Session()
        self.Komoot = requests.Session()
        self.USCensus = requests.Session()

def create_sessions():
    return GeoSessions()

def geocode_address(address, s):
    g = geocoder.komoot(address, session=s.Komoot)
    if (g.ok == False):
        g = geocoder.arcgis(address, session=s.Arcgis)
    if (g.ok == False):
        g = geocoder.uscensus(address, session=s.USCensus)
    return g

In [70]:
s = create_sessions()

In [71]:
addr = addrs_df['address'][0] + ', Chicago, IL'
tmp = geocode_address(addr, s)
print(addr)
print('Latitude:  {}'.format(tmp.lat))
print('Longitude: {}'.format(tmp.lng))

2100 s archer av, Chicago, IL
Latitude:  41.85426174412816
Longitude: -87.63198586874586


In [72]:
addr = addrs_df['address'][0] + ', Chicago, IL'
tmp = geocoder.uscensus(addr, session=s.USCensus)
print(addr)
print('Latitude:  {}'.format(tmp.lat))
print('Longitude: {}'.format(tmp.lng))

2100 s archer av, Chicago, IL
Latitude:  41.85358
Longitude: -87.63335


In [73]:
addr = addrs_df['address'][0] + ', Chicago, IL'
tmp = geocoder.komoot(addr, session=s.Komoot)
print(addr)
print('Latitude:  {}'.format(tmp.lat))
print('Longitude: {}'.format(tmp.lng))

2100 s archer av, Chicago, IL
Latitude:  41.85455545
Longitude: -87.63227488755658


In [74]:
OUT_PATH = os.path.join('data', 'hacknight_sample_data_geocode.csv')
if os.path.isfile(OUT_PATH):
    addrs_df = pd.read_csv(OUT_PATH)
    print('It existed')
addrs_df.head()

It existed


,Unnamed: 0,address,lat,lng
0,0,2100 s archer av,41.854262,-87.631986
1,1,400 n laramie,41.887918,-87.755411
2,2,1300 w cornelia,41.945332,-87.661621
3,3,200 w cermak rd,41.852951,-87.632015
4,4,2400 n monticello av,41.924665,-87.718253


In [75]:
addrs_df.drop('Unnamed: 0', axis=1, inplace=True)
addrs_df.head()

,address,lat,lng
0,2100 s archer av,41.854262,-87.631986
1,400 n laramie,41.887918,-87.755411
2,1300 w cornelia,41.945332,-87.661621
3,200 w cermak rd,41.852951,-87.632015
4,2400 n monticello av,41.924665,-87.718253


In [20]:
failed_inds = []
iterations_between_saves = 100

I kept rerunning the cell below, waiting for something to blow up. Nothing blew up though. This geocoded about 100 addresses per minute, it didn't require an API key, and the geocoder documentation doesn't list any restrictions on the use of the data.

In [78]:
missing_latlong_indices = addrs_df[addrs_df['lat'].isnull()][:2001].index.tolist()

for i in missing_latlong_indices:
    try:
        if i%iterations_between_saves == 0:
            print('reached index {}, saving df'.format(i))
            addrs_df.to_csv(OUT_PATH, index=False)
        addr = addrs_df['address'][i] + ', Chicago, IL'
        tmp = geocode_address(addr, s)
        addrs_df.loc[i,'lat'] = tmp.lat
        addrs_df.loc[i,'lng'] = tmp.lng
    except Exception as e:
        failed_inds.append(i)
addrs_df.to_csv(OUT_PATH, index=False)

reached index 19800, saving df
reached index 19900, saving df
reached index 20000, saving df
reached index 20100, saving df
reached index 20200, saving df
reached index 20300, saving df
reached index 20400, saving df
reached index 20500, saving df
reached index 20600, saving df


In [86]:
addrs_df.to_csv(OUT_PATH, index=False)

In [88]:
addrs_df.iloc[-25:]

,address,lat,lng
20583,4300 s talman,41.815545,-87.690810
20584,2800 w cortez,41.900575,-87.696901
20585,3500 s bell,41.830372,-87.681430
20586,1600 n sedgwick,41.911148,-87.638639
20587,5700 s michigan ave,41.790842,-87.622408
20588,2700 w maypole,41.883677,-87.693992
20589,7200 s constance ave,41.764226,-87.580604
20590,2800 e 77th pl,41.754561,-87.555020
20591,3400 s prairie,41.832746,-87.620598
20592,10400 s calumet,41.705333,-87.614970


In [80]:
failed_inds

[]

In [91]:
addrs_df['address'] = addrs_df['address'] + ', chicago, il'

In [97]:
geocoded_df = pd.merge(left=df, right=addrs_df, how='inner', on='address')
geocoded_df.head()

,ticket_number,issue_date,violation_location,license_plate_number,license_plate_state,license_plate_type,zipcode,violation_code,violation_description,unit,unit_description,vehicle_make,fine_level1_amount,fine_level2_amount,current_amount_due,total_payments,ticket_queue,ticket_queue_date,notice_level,hearing_disposition,notice_number,officer,address,lat,lng
0,9188814621,2015-05-07 13:52:00,2134 S ARCHER AV,7c189a16ef79db9413c1f46b7e5d1712e5c0c1575be352...,MI,PAS,48103,0964190A,EXP. METER NON-CENTRAL BUSINESS DISTRICT,498,DOF,BUIC,50,100,0.0,50.0,Paid,2015-05-13,NaN,NaN,0,798,"2100 s archer av, chicago, il",41.854262,-87.631986
1,9188417037,2015-01-01 21:29:00,2167 S ARCHER AV,ac7f6f4be49dfa223571838e586653aa5cdcb6bdfc6385...,IL,PAS,NaN,0964190A,EXP. METER NON-CENTRAL BUSINESS DISTRICT,502,DOF,MITS,50,100,0.0,50.0,Paid,2015-01-05,NaN,NaN,0,1529,"2100 s archer av, chicago, il",41.854262,-87.631986
2,9188598734,2015-03-26 13:14:00,2140 S ARCHER AV,a493850f62c8ca5a25787271859f0409c289fb84130f33...,OH,PAS,45409,0976160A,REAR AND FRONT PLATE REQUIRED,498,DOF,PORS,60,120,146.4,0.0,Notice,2015-04-08,SEIZ,NaN,5191566530,780,"2100 s archer av, chicago, il",41.854262,-87.631986
3,9189154639,2015-07-24 14:18:00,2172 S ARCHER AV,38a1ef0c0a325c79e9dba69f6e96652fcbf1e9dcb5db3f...,IL,PAS,606161514,0964190A,EXP. METER NON-CENTRAL BUSINESS DISTRICT,498,DOF,MERZ,50,100,0.0,0.0,Dismissed,2015-09-14,VIOL,Not Liable,5176435360,796,"2100 s archer av, chicago, il",41.854262,-87.631986
4,9188041623,2015-01-17 14:45:00,2107 S ARCHER AV,e3c39cce6c30735cc977656a463233ea6f516328828e8d...,LA,PAS,70072,0964190A,EXP. METER NON-CENTRAL BUSINESS DISTRICT,502,DOF,HOND,50,100,122.0,0.0,Notice,2015-02-04,SEIZ,NaN,5181845840,1461,"2100 s archer av, chicago, il",41.854262,-87.631986
